In [1]:
import os
import sys

from dotenv import load_dotenv

pkg_path = os.path.abspath("../")
if pkg_path not in sys.path:
    sys.path.append(pkg_path)

load_dotenv()

True

## Criação do agente


Criação do agente especialista em filmes como checkpoints salvos na memória de trabalho (`MemorySaver`).

A orquestração das ações se baseia no modelo `gpt-4o` da OpenAI.

In [2]:
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from src.agent import ReactAgent

llm_agent = ChatOpenAI(model="gpt-4o", temperature=0)
agent = ReactAgent(llm=llm_agent, memory=MemorySaver())

bolt://localhost:7687


## Protótipo de interface - Gradio

[Gradio](https://www.gradio.app) é um pacote Python de código aberto que permite a criação de aplicativos web para modelos de machine learning, API ou função Python. Além disso, é pssível compartilhar a aplicação criada por meio de um link público.

In [3]:
!pip install gradio -q

In [4]:
# Front end web app for the assistant
import gradio as gr

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")
    chat_history = []
    
    def user(user_message, history):
        answer = agent.answer_question(question=user_message,  thread_id="1")
        history.append((user_message, answer["response"]))
       
        return gr.update(value=""), history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False)
    clear.click(lambda: None, None, chatbot, queue=False)

if __name__ == "__main__":
    demo.launch(debug=True)

/opt/miniconda3/envs/llm-agent/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


## Requsição assíncrona à API em FastAPI

In [2]:
import httpx

async def make_request(query: str, session_id: str):
    url = 'http://localhost:8000/answer'
    params = {
        'query': query,
        'session_id': session_id
    }
    headers = {
        'accept': 'application/json',
        'x-api-key': os.getenv('ASSISTANT_API_KEY')
    }
    
    async with httpx.AsyncClient() as client:
        response = await client.get(url, params=params, headers=headers)
        return response.json()


In [7]:
answer = await make_request(query="Atores que trabalharam com Arndold Schazenerger", session_id="1")
print(answer)

{'query': 'Atores que trabalharam com Arndold Schazenerger', 'response': 'Arnold Schwarzenegger trabalhou com diversos atores ao longo de sua carreira. Alguns dos atores que atuaram ao lado dele incluem:\n\n- Chopper Bernet\n- Earl Boen\n- Mark Famiglietti\n- Kristanna Loken\n- David Andrews\n- Claire Danes\n- Nick Stahl\n- Xander Berkeley\n- Jenette Goldstein\n- Danny Cooksey\n\nSe precisar de informações sobre algum filme específico ou outros detalhes, estou à disposição!', 'reasoning': [{'action': 'ai', 'observation': '================================== Ai Message ==================================\nTool Calls:\n  get_info_about_movies (call_yznpOlPmbSkmHOHLxyJ4OeDr)\n Call ID: call_yznpOlPmbSkmHOHLxyJ4OeDr\n  Args:\n    query: atores que trabalharam com Arnold Schwarzenegger'}, {'action': 'tool', 'observation': '================================= Tool Message =================================\nName: get_info_about_movies\n\n{"query": "atores que trabalharam com Arnold Schwarzenegger